In [17]:
import sqlite3
import os
import pandas as pd
import xml.etree.ElementTree as ET

"""results 파일에서 시뮬레이션 횟수 만큼 엑셀 변환"""
# ✅ Excel 파일이 있는 폴더 경로
folder_path = r"C:\VISSIM_Workspace\network_final"
file_path = r"C:\VISSIM_Workspace\network_final\network01.inpx"

from_time = 1800
siml_count = 10

# XML 파일 파싱
tree = ET.parse(file_path)
root = tree.getroot()

for elem in root.iter("dataColl"):
    interval_value = int(elem.get("interval"))  # "interval" 속성 값 가져오기

results_path = folder_path + "\\" + "network01.results"
results_files = sorted([f for f in os.listdir(results_path) if f.endswith(".db")], key=lambda x: int(x.split(".")[0]))

# 10개씩 묶어서 처리
for start in range(int(len(results_files)/2), len(results_files), siml_count):
    batch_files = results_files[start:start + siml_count]  # 10개 단위로 분할
    print("batch files : ", batch_files)
    df_list = []
    for i in range(len(batch_files)):
        db_path = os.path.join(results_path, batch_files[i])
        print("db_path : ", db_path)
        # 데이터베이스 연결

        with sqlite3.connect(db_path) as conn:
            df = pd.read_sql_query("SELECT * FROM DATACOLLECTIONMEASUREMENT_EvaluationTimeIntervalClass;", conn)
        df_list.append(df)
    df = pd.concat(df_list).groupby(level=0).mean()
    print("df : ", df.head(5))

    df[["ARG_TIMEINTERVAL", "OBJECT_ID"]] = df_list[0][["ARG_TIMEINTERVAL", "OBJECT_ID"]]

    # TIMEINT 변환
    df["ARG_TIMEINTERVAL"] = from_time + df["ARG_TIMEINTERVAL"].astype(int) * interval_value   # 변환 시작점 보정

    # OCCUPRATE 보정
    df["OCCUPRATE"] = df["OCCUPRATE"].astype(float)*100
    df["OCCUPRATE"] = df["OCCUPRATE"].apply(lambda x: f"{x:.2f}")  # 항상 소수점 둘째 자리까지 유지
    df = df.round(2)
    df["OCCUPRATE"] = df["OCCUPRATE"].astype(str) + "%"

    # SIMRUN 설정
    df["SIMRUN"] =siml_count
    df["OBJECT_ID"] = df["OBJECT_ID"].astype(int)
    # 컬럼명 변경
    df.rename(columns={
        "ARG_TIMEINTERVAL": "TIMEINT",
        "OBJECT_ID": "DATACOLLECTIONMEASUREMENT",
        "ACCELERATION" : "ACCELERATION(ALL)",
        "DIST" : "DIST(ALL)",
        "LENGTH" : "LENGTH(ALL)",
        "VEHS" : "VEHS(ALL)",
        "PERS" : "PERS(ALL)",
        "QUEUEDELAY" : "QUEUEDELAY(ALL)",
        "SPEEDAVGARITH" : "SPEEDAVGARITH(ALL)",
        "SPEEDAVGHARM" : "SPEEDAVGHARM(ALL)",
        "OCCUPRATE" : "OCCUPRATE(ALL)"
    }, inplace=True)

    # 순서 재정의
    df = df[["SIMRUN", "TIMEINT", "DATACOLLECTIONMEASUREMENT", "ACCELERATION(ALL)", "DIST(ALL)",
             "LENGTH(ALL)", "VEHS(ALL)", "PERS(ALL)", "QUEUEDELAY(ALL)", "SPEEDAVGARITH(ALL)", "SPEEDAVGHARM(ALL)", "OCCUPRATE(ALL)"]]

    excel_dir  = os.path.join(folder_path, "network02_output")
    if not os.path.exists(excel_dir):
        os.makedirs(excel_dir)
    last_file = xlsx_files = sorted([f for f in os.listdir(excel_dir) if f.endswith(".xlsx")], key=lambda x: int(x.split("_")[0]) if x.split("_")[0].isdigit() else 0)

    if not last_file:
        last_file = 1
    else:
        last_file = int(xlsx_files[-1].split("_")[0]) + 1  # 가장 마지막 파일 번호 + 1
    excel_path = os.path.join(excel_dir, f"{last_file}_output.xlsx")
    df.to_excel(excel_path, index=False)
    print(f"생성파일: {excel_path}")

batch files :  ['241.db', '242.db', '243.db', '244.db', '245.db', '246.db', '247.db', '248.db', '249.db', '250.db']
db_path :  C:\VISSIM_Workspace\network_final\network01.results\241.db
db_path :  C:\VISSIM_Workspace\network_final\network01.results\242.db
db_path :  C:\VISSIM_Workspace\network_final\network01.results\243.db
db_path :  C:\VISSIM_Workspace\network_final\network01.results\244.db
db_path :  C:\VISSIM_Workspace\network_final\network01.results\245.db
db_path :  C:\VISSIM_Workspace\network_final\network01.results\246.db
db_path :  C:\VISSIM_Workspace\network_final\network01.results\247.db
db_path :  C:\VISSIM_Workspace\network_final\network01.results\248.db
db_path :  C:\VISSIM_Workspace\network_final\network01.results\249.db
db_path :  C:\VISSIM_Workspace\network_final\network01.results\250.db
df :     ACCELERATION        DIST    LENGTH  VEHS  PERS    OCCUPTM  QUEUEDELAY  \
0     -0.285841    1.529259  5.552846  71.5  71.5  31.829518    0.000000   
1      0.125174  101.44076

파일 이름 변경 및 재저장 완료!
